## Goal - Merge several shapefiles using OGR2OGR
##      - Project the result to PT-TM06/ETRS89

In [ ]:
import os
from osgeo import ogr

### Paths to data

In [ ]:
datafolder = '/home/gisuser/cwgis/demo5gdal/'
outfile    = '/home/gisuser/cwgis/results/demo5gdal.shp'

In [ ]:
import os

In [ ]:
def list_esri_shp(datafolder):
    """
    List ESRI Shapefiles in Folder
    """

    files = []
    for (d, _d, f) in os.walk(datafolder):
        files.extend(f)
        break

    return [f for f in files if f.split('.')[-1] == 'shp']

In [ ]:
"""
GDAL Drivers Name
"""

def drv_name(_file):
    """
    Return the driver for a given file format
    """
    
    drv = {
        # Vector files
        '.gml'    : 'GML',
        '.shp'    : 'ESRI Shapefile',
        '.json'   : 'GeoJSON',
        '.kml'    : 'KML',
        '.osm'    : 'OSM',
        '.dbf'    : 'ESRI Shapefile',
        '.vct'    : 'Idrisi',
        '.nc'     : 'netCDF',
        '.vrt'    : 'VRT',
        '.mem'    : 'MEMORY',
        '.sqlite' : 'SQLite',
        '.gdb'    : 'FileGDB',
        # Raster files
        '.tif'    : 'GTiff',
        '.ecw'    : 'ECW',
        '.mpr'    : 'ILWIS',
        '.mpl'    : 'ILWIS',
        '.jpg'    : 'JPEG',
        '.nc'     : 'netCDF',
        '.png'    : 'PNG',
        '.vrt'    : 'VRT',
        '.asc'    : 'AAIGrid',
        '.img'    : 'HFA',
        # Vector or Raster
        '.gpkg'   : 'GPKG'
    }
    
    return str(drv[os.path.splitext(_file)[1]])

In [ ]:
def execmd(cmd):
    """
    Execute a command and provide information about the results
    """
    import subprocess
    
    p = subprocess.Popen(cmd, shell=True,
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    out, err = p.communicate()
    
    if p.returncode != 0:
        print(cmd)
        raise ValueError((
            'Message: Command execution ended with error\n'
            'Command was: {cmd}\n'
            'Output: {o}\n'
            'Error: {e}'
        ).format(
            cmd=cmd, o=out.decode('utf-8'), e=err.decode('utf-8')
        ))
    
    else:
        return out.decode('utf-8')

## List all files in data folder

In [ ]:
shps = list_esri_shp(datafolder)

In [ ]:
shps

### Merge all shapefiles into one

In [ ]:
tmpshp = os.path.join(
    os.path.dirname(outfile),
    f'tmp_{os.path.basename(outfile)}'
)

out_drv = drv_name(tmpshp)

for i in range(len(shps)):
    if not i:
        # Create output and copy some features of one layer (first in shps)
        cmd = 'ogr2ogr -f "{}" {} {}'.format(
            out_drv, tmpshp, os.path.join(datafolder, shps[i])
        )
    
    else:
        cmd = 'ogr2ogr -f "{}" -update -append {} {}'.format(
            out_drv, tmpshp, os.path.join(datafolder, shps[i])
        )
        
    cmdout = execmd(cmd)

### Project to ETRS89

In [ ]:
data = ogr.GetDriverByName(drv_name(tmpshp)).Open(tmpshp)

lyr   = data.GetLayer() 
spref = lyr.GetSpatialRef()

epsg = int(str(spref.GetAttrValue('AUTHORITY', 1)))
print(epsg)

cmd = 'ogr2ogr -f "{}" {} {} -s_srs EPSG:{} -t_srs EPSG:{}'.format(
    drv_name(outfile), outfile, tmpshp,
    str(epsg), str(3763)
)

outcmd = execmd(cmd)